In [ ]:
import json
import socket
HOST = ''              # Endereco IP do Servidor
PORT = 5000            # Porta que o Servidor esta

In [ ]:
class GameServer:

    def __init__(self, number_rows, row_size):
        self.number_rows = number_rows
        self.row_size = row_size
        self.whos_turn = 1
        self.is_game_over = False
        
        self.board = []
        for i in range(number_rows):
            self.board.append(row_size)
        
        self._s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self._s.bind((HOST, PORT))
        self._s.listen(2)
        
        self._init_players()
        
    def _init_players(self):
        self.players = []
        
        con1, cli1 = self._s.accept()
        con2, cli2 = self._s.accept()
        
        player_info_1 = self.recv_data(con1)
        player_info_2 = self.recv_data(con2)
        
        player_info_1['con'] = con1
        player_info_2['con'] = con2
        
        self.players.extend([player_info_1, player_info_2])
        
        
    def recv_data(self, con):
        return json.loads(self._recv(con))
        
    def send_data(self, resp_data, con):
        self._send(bytes(json.dumps(resp_data), 'UTF-8'), con)
        
    def _send(msg, con):
        con.send(bytes(msg, 'UTF-8'))
        
    def _recv(con):
        return repr(con.recv(1024))
        
    def finish(self):
        for player in self.players:
            game_data = {
                'is_game_over': self.is_game_over,
                'winner': self.players[self.whos_turn]['name']
            }
            self.send_data(game_data, player['con'])
            player['con'].close()
            
    def print_board(self):
        for row_idx in range(self.number_rows):
            print('Row {}: '.format(row_idx) + self.board[row_idx]*'@')
            print('\n')
            
    def not_empty_rows(self):
        not_empty_rows = []
        for idx in range(self.number_rows):
            if self.board[idx] > 0:
                not_empty_rows.append(idx)
        return not_empty_rows
            
    def is_not_over(self):
        if len(self.not_empty_rows()) < 1:
            return False
        self.is_game_over = True
        return True
    
    def turn(self):
        
        # Manda a informacao do jogo para o jogador
        game_data = {
            'is_game_over': self.is_game_over,
            'turns_player': self.players[self.whos_turn]['name'],
            'board': self.board,
            'not_empty_rows': self.not_empty_rows()
        }
        self.send_data(game_data, self.players[self.whos_turn]['con'])
        
        # Recebe informacoes da jogada do jogador
        resp_data = self.recv_data(self.players[self.whos_turn]['con'])
        
        # Imprime a jogada do Jogador
        print('Jogada de {}: '.format(self.players[self.whos_turn]['name']))
        print('Grupo {}'.format(resp_data['row']))
        print('Pecas {}'.format(resp_data['pebles']))
        
        # Imprime o tabuleiro resultante da jogada
        self.print_board()
            
    def main(self):
        self.print_board()
        
        while(self.is_not_over()):
            self.whos_turn = 1 - self.whos_turn
            self.turn()
            
        self.finish()